In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu copy


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_co2(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * 10
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
gridSize = DiscreteParameter('gridSize', domain=[3, 4, 5, 6, 7, 8, 9, 10])
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=8, max_value=20)  # 28.8 km/h to 90 km/h
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1, 2, 3])
accel = ContinuousParameter('accel', 1., 3.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, numberOfLanes, accel])

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [9]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_co2(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_co2(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(250)
train_Y = user_function_co2(train_X)

[[ 6.         19.02017817  1.          1.31260122]
 [ 5.         16.49778617  1.          1.72897976]
 [ 7.         13.74056629  3.          2.59172409]
 [ 8.          9.69058353  3.          1.19613451]
 [ 5.         14.44633812  3.          2.06263358]
 [ 8.         18.89744143  3.          1.92112732]
 [ 5.         19.6047041   3.          1.84365678]
 [10.         12.12637465  3.          2.78286927]
 [ 8.         15.05442344  2.          1.70181205]
 [ 6.         16.7079148   2.          1.83815463]
 [ 3.         12.37940295  3.          2.62570956]
 [ 6.         17.35444002  2.          1.0341496 ]
 [ 7.         10.05389895  1.          2.89729229]
 [ 6.          9.48228134  3.          2.95066113]
 [ 8.         13.81777898  3.          2.27152095]
 [ 8.         10.30946286  2.          2.38940478]
 [ 5.         10.22267967  3.          1.45374008]
 [ 4.         15.34765972  2.          1.76085092]
 [ 5.         10.72628409  1.          1.94130258]
 [ 5.         16.13288434  3.  

 Retrying in 1 seconds

Evaluating input: 2 of 250

 Retrying in 1 seconds

Evaluating input: 3 of 250

 Retrying in 1 seconds

Evaluating input: 4 of 250

 Retrying in 1 seconds

Evaluating input: 5 of 250

 Retrying in 1 seconds

Evaluating input: 6 of 250

 Retrying in 1 seconds

Evaluating input: 7 of 250

 Retrying in 1 seconds

Evaluating input: 8 of 250

 Retrying in 1 seconds

Evaluating input: 9 of 250

 Retrying in 1 seconds

Evaluating input: 10 of 250

 Retrying in 1 seconds

Evaluating input: 11 of 250

 Retrying in 1 seconds

Evaluating input: 12 of 250

 Retrying in 1 seconds

Evaluating input: 13 of 250

 Retrying in 1 seconds

Evaluating input: 14 of 250

 Retrying in 1 seconds

Evaluating input: 15 of 250

 Retrying in 1 seconds

Evaluating input: 16 of 250

 Retrying in 1 seconds

Evaluating input: 17 of 250

 Retrying in 1 seconds

Evaluating input: 18 of 250

 Retrying in 1 seconds

Evaluating input: 19 of 250

 Retrying in 1 seconds

Evaluating input: 20 of 250

 

 Retrying in 1 seconds

Evaluating input: 156 of 250

 Retrying in 1 seconds

Evaluating input: 157 of 250

 Retrying in 1 seconds

Evaluating input: 158 of 250

 Retrying in 1 seconds

Evaluating input: 159 of 250

 Retrying in 1 seconds

Evaluating input: 160 of 250

 Retrying in 1 seconds

Evaluating input: 161 of 250

 Retrying in 1 seconds

Evaluating input: 162 of 250

 Retrying in 1 seconds

Evaluating input: 163 of 250

 Retrying in 1 seconds

Evaluating input: 164 of 250

 Retrying in 1 seconds

Evaluating input: 165 of 250

 Retrying in 1 seconds

Evaluating input: 166 of 250

 Retrying in 1 seconds

Evaluating input: 167 of 250

 Retrying in 1 seconds

Evaluating input: 168 of 250

 Retrying in 1 seconds

Evaluating input: 169 of 250

 Retrying in 1 seconds

Evaluating input: 170 of 250

 Retrying in 1 seconds

Evaluating input: 171 of 250

 Retrying in 1 seconds

Evaluating input: 172 of 250

 Retrying in 1 seconds

Evaluating input: 173 of 250

 Retrying in 1 seconds

Eval

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [10]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = 94.28493209156616
Optimization restart 2/5, f = 94.28493210143097
Optimization restart 3/5, f = 94.28493209120934
Optimization restart 4/5, f = 94.28493209704962
Optimization restart 5/5, f = 94.28493218443478


GP_regression.,value,constraints,priors
rbf.variance,11167.623698052479,+ve,
rbf.lengthscale,12.812915597026457,+ve,
Gaussian_noise.variance,332.95690627873626,+ve,


#### Acquisition Function

In [11]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [12]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [13]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    update_interval = 1,
    batch_size = 5
)

In [14]:
ed_loop.run_loop(user_function_co2, 50)

Optimization restart 1/5, f = 94.28493209120934
Optimization restart 2/5, f = 94.28493209120747
Optimization restart 3/5, f = 94.28493209178791
Optimization restart 4/5, f = 94.28493209126802
Optimization restart 5/5, f = 98.16130356665495


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


[[10. 20.  1.  3.]
 [ 3.  8.  1.  1.]
 [10.  8.  1.  1.]
 [ 3. 20.  1.  3.]
 [10. 20.  1.  1.]]

User function time loss called with 5 inputs to simulate

Evaluating input: 1 of 5

 Retrying in 1 seconds

Evaluating input: 2 of 5

 Retrying in 1 seconds

Evaluating input: 3 of 5

 Retrying in 1 seconds

Evaluating input: 4 of 5

 Retrying in 1 seconds

Evaluating input: 5 of 5

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 1/5, f = 122.08807990759045
Optimization restart 2/5, f = 122.08807209097736
Optimization restart 3/5, f = 122.08807209050556
Optimization restart 4/5, f = 131.42163913619584
Optimization restart 5/5, f = 122.08807212451238
[[ 3. 20.  3.  1.]
 [ 3.  8.  3.  3.]
 [10.  8.  3.  1.]
 [10. 20.  3.  3.]
 [10.  8.  1.  3.]]

User function time loss called with 5 inputs to simulate

Evaluating input: 1 of 5

 Retrying in 1 seconds

Evaluating input: 2 of 5

 Retrying in 1 seconds

Evaluating input: 3 of 5

 Retrying in 1 seconds

Evaluating input: 4 of 5

 Retrying in 1 seconds

Evaluating input: 5 of 5

 Retrying in 1 seconds


...

Optimization restart 1/5, f = 1061.9498561003136
Optimization restart 2/5, f = 1088.6573101948802
Optimization restart 3/5, f = 1088.657310197989
Optimization restart 4/5, f = 1088.6573101918057
Optimization restart 5/5, f = 1088.6573159228255
[[ 9.         17.05353359  1.          1.91201651]
 [10.         10.50268733  2.          1.74179157]
 [ 5.         18.19201363  1.          2.02165067]
 [ 9.         19.42010661  2.          3.        ]
 [ 7.         11.53023639  2.          3.        ]]

User function time loss called with 5 inputs to simulate

Evaluating input: 1 of 5

 Retrying in 1 seconds

Evaluating input: 2 of 5

 Retrying in 1 seconds

Evaluating input: 3 of 5

 Retrying in 1 seconds

Evaluating input: 4 of 5

 Retrying in 1 seconds

Evaluating input: 5 of 5

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1085.0060696704422
Optimization restart 2/5, f = 1108.0195078573688
Optimization restart 3/5, f = 1108.0195078555885
Optimization restart 4/5, f = 1108.0195078512718
Optimization restart 5/5, f = 1108.0195078512052
[[ 8.         20.          1.          2.0968396 ]
 [ 9.         18.70020869  3.          1.        ]
 [ 4.         17.32103883  2.          3.        ]
 [ 5.         20.          3.          1.        ]
 [10.          8.          2.          1.90728634]]

User function time loss called with 5 inputs to simulate

Evaluating input: 1 of 5

 Retrying in 1 seconds

Evaluating input: 2 of 5

 Retrying in 1 seconds

Evaluating input: 3 of 5

 Retrying in 1 seconds

Evaluating input: 4 of 5

 Retrying in 1 seconds

Evaluating input: 5 of 5

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1105.7424864437678
Optimization restart 2/5, f = 1131.1336206738315
Optimization restart 3/5, f = 1131.1336206756853
Optimization restart 4/5, f = 1131.1336206733947
Optimization restart 5/5, f = 1131.1336206738379


### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [19]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = 105.51770371225868
Optimization restart 2/5, f = 94.28493209147018
Optimization restart 3/5, f = 94.28494224818732
Optimization restart 4/5, f = 94.28493209141652
Optimization restart 5/5, f = 94.2849324900023


#### Acquisition Function

In [20]:
integrated_variance_reduction = IntegratedVarianceReduction(model=emukit_model_ird, space=parameter_space)

#### Optimizer

In [21]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [22]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    update_interval = 1,
    batch_size = 10
)

In [23]:
ed_loop.run_loop(user_function_co2, 10)

Optimization restart 1/5, f = 94.28493209141652
Optimization restart 2/5, f = 94.28493215235721
Optimization restart 3/5, f = 98.1627073518468
Optimization restart 4/5, f = 98.16124984627456
Optimization restart 5/5, f = 97.53944354199507


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


[[10.         19.649408    1.          2.82442587]
 [10.          8.          1.          1.        ]
 [ 3.          8.          1.          1.        ]
 [ 3.         20.          1.          3.        ]
 [10.         15.18994624  1.          1.        ]
 [10.         20.          3.          1.        ]
 [ 3.          8.          1.          3.        ]
 [10.         14.51663953  1.          3.        ]
 [10.          8.          3.          3.        ]
 [ 3.         20.          3.          1.        ]]

User function time loss called with 10 inputs to simulate

Evaluating input: 1 of 10

 Retrying in 1 seconds

Evaluating input: 2 of 10

 Retrying in 1 seconds

Evaluating input: 3 of 10

 Retrying in 1 seconds

Evaluating input: 4 of 10

 Retrying in 1 seconds

Evaluating input: 5 of 10

 Retrying in 1 seconds

Evaluating input: 6 of 10

 Retrying in 1 seconds

Evaluating input: 7 of 10

 Retrying in 1 seconds

Evaluating input: 8 of 10

 Retrying in 1 seconds

Evaluating input: 9 o

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 143.559847390387
Optimization restart 2/5, f = 143.5598473900277
Optimization restart 3/5, f = 143.55984739116525
Optimization restart 4/5, f = 156.53992937810838
Optimization restart 5/5, f = 143.55984739636386


...

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 4/5, f = 563.883577606943
Optimization restart 5/5, f = 443.96461361248
[[ 8.         17.3390901   2.          2.07169948]
 [ 4.         15.27772899  1.          1.86095564]
 [ 8.         13.79482997  3.          1.        ]
 [ 8.         11.82501342  1.          3.        ]
 [ 3.         10.76249101  3.          1.73714911]
 [ 6.         20.          3.          1.91452215]
 [10.         16.84717963  1.          2.49942394]
 [10.         11.39329005  2.          1.82824423]
 [ 5.         15.05058632  3.          3.        ]
 [ 5.         10.42278225  2.          2.09441223]]

User function time loss called with 10 inputs to simulate

Evaluating input: 1 of 10

 Retrying in 1 seconds

Evaluating input: 2 of 10

 Retrying in 1 seconds

Evaluating input: 3 of 10

 Retrying in 1 seconds

Evaluating input: 4 of 10

 Retrying in 1 seconds

Evaluating input: 5 of 10

 Retrying in 1 seconds

Evaluating input: 6 of 10

 Retrying in 1 seconds

Evaluating input: 7 of 10

 Re

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 486.87883042983054
Optimization restart 2/5, f = 486.8517483024623
Optimization restart 3/5, f = 486.8517483088396
Optimization restart 4/5, f = 486.8517483025067
Optimization restart 5/5, f = 604.8805271200446


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


### Random GP

Supervised learning

In [15]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=5)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/5, f = 1054.7799404405848
Optimization restart 2/5, f = 1054.7799391494045
Optimization restart 3/5, f = 1054.779939149284
Optimization restart 4/5, f = 1054.7799393865603
Optimization restart 5/5, f = 1054.7799391493022


GP_regression.,value,constraints,priors
rbf.variance,18884.350833241704,+ve,
rbf.lengthscale,6.464151373280042,+ve,
Gaussian_noise.variance,183.47908319576428,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [16]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  8.970729554163452


In [24]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  13.264605946984197


In [17]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  13.405015625476608


In [18]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is 33.079% better than training on random points.'

In [25]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is 1.047% better than training on random points.'

### Save Models

In [28]:
with open(f'experimental_design/models/model_variance_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design/models/integrated_variance_reduction_20_init_points_10_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design/models/250_random_points_supervised_GP_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)